--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine.  There are lots of numbers and symbols you don't really understand, but apparently
1. any number adjacent to a symbol, even diagonally, is a "part number"
1. and should be included in your sum.
1. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol:
- 114 (top right)
- 58 (middle right).

Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [61]:
import numpy as np
with open("files/day_03.txt") as input_file:
    map = [list(line.strip()) for line in input_file.readlines()]

map_int = np.zeros((len(map), len(map[0])), dtype=bool)
map_symbol = np.zeros((len(map), len(map[0])), dtype=bool)
for y in range(len(map)):
    for x in range(len(map[y])):
        if map[y][x] in "0123456789":
            map_int[y][x] = 1
        elif map[y][x] != ".":
            map_symbol[y][x] = 1

numbers = []
max_y = len(map_int)
max_x = len(map_int[0])
for y in range(max_y):
    current_number = ""
    symbol_found = False
    for x in range(max_x):
        if map_int[y][x]:
            if not symbol_found:
                #Check behind
                if current_number == "":
                    if y-1 >= 0 and x-1 >= 0 and map_symbol[y-1][x-1]:
                        symbol_found = True
                    elif x-1 >= 0 and map_symbol[y][x-1]:
                        symbol_found = True
                    elif y+1 < max_y and x-1 >= 0 and map_symbol[y+1][x-1]:
                        symbol_found = True

                if y-1 >= 0 and map_symbol[y-1][x]:
                    symbol_found = True
                elif y+1 < max_y and map_symbol[y+1][x]:
                    symbol_found = True
            current_number += map[y][x]

        if (map_int[y][x] == 0 or x+1==max_x) and current_number != "":
            if not symbol_found:
                if y-1 >= 0 and map_symbol[y-1][x]:
                    symbol_found = True
                elif map_symbol[y][x]:
                    symbol_found = True
                elif y+1 < max_y and map_symbol[y+1][x]:
                    symbol_found = True
            if symbol_found:
                numbers.append(int(current_number))
                symbol_found = False
            current_number = ""
print("Answer:", sum(numbers))

Answer: 538046


--- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong.
- A gear is any * symbol that is adjacent to exactly two part numbers.
- Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [38]:
import numpy as np
with open("files/day_03.txt") as input_file:
    map = [list(line.strip()) for line in input_file.readlines()]

map_int = np.zeros((len(map), len(map[0])), dtype=int)
map_gear = np.zeros((len(map), len(map[0])), dtype=int)
for y in range(len(map)):
    for x in range(len(map[y])):
        if map[y][x] == "*":
            map_gear[y][x] = 1
        elif map[y][x] in "0123456789":
            map_int[y][x] = 1


max_y = len(map_int)
max_x = len(map_int[0])
numbers = []
for y in range(max_y):
    for x in range(max_x):
        if not map_gear[y][x]:
            continue
        sub_map = [map_int[y+dy][x+dx] for dy in range(-1,2) for dx in range(-1,2)]
        print(sub_map)
        adjacent_numbers = 0
        above = sub_map[0:3]
        if sum(above) == 3 or above == [1,1,0] or above == [0,1,1] or sum(above) == 1:
            int_line = map_int[y-1].tolist()
            index = int_line[x-1 if x-1>=0 else 0:x+1 if x+1 < max_x else max_y].index(1)
            print("1 above", index)
            print(int_line[x-1:x+2])
            print(int_line[:x+index:-1])
            number = map[y][x - int_line[:x+index:-1] : int_line.index(0, x+index)]
            print("\tFUICK", int_line[x+index], number)
            adjacent_numbers += 1
            
            number = map[y-1][int_line[:x+1]]
        elif above == [1,0,1]:
            print("2 above")
            adjacent_numbers += 2

        below = sub_map[6:10]
        if below == [1,1,0] or below == [0,1,1] or sum(below) == 1:
            print("1 below")
            adjacent_numbers += 1
        elif below == [1,0,1]:
            print("2 below")
            adjacent_numbers += 2
        
        if sub_map[3] == 1:
            print("1 left")
            adjacent_numbers += 1
            number = "".join([val for val in map[y][:x]])
            numbers.append(int(number[number.rfind(".")+1 if "." in number else 0:]))
        if sub_map[5] == 1:
            print("1 right")
            number = "".join([val for val in map[y][x+1:]])
            numbers.append(int(number[:number.find(".") if "." in number else -1]))
            adjacent_numbers += 1

        if adjacent_numbers < 2:
            continue
        print(f"gear at {x},{y} have {adjacent_numbers} adjacent numbers")


[1, 0, 0, 0, 0, 0, 1, 1, 0]
1 above -1
[1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
	FUICK 1 []


TypeError: list indices must be integers or slices, not list